2.可视化卷积神经网络的过滤器：
在输入空间中进行梯度上升来实现：从输入空白输入图像开始，将梯度下降应用于输出值，让某个过滤器的相应最大化

In [1]:
from keras import models
from keras.applications import vgg16
from keras import backend as K

model=vgg16.VGG16(weights='imagenet',include_top=False)
print(model.summary())

2021-12-11 22:10:41.285666: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-11 22:10:41.760466: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8901 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [2]:
layer_name='block3_conv1'
filter_index=0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:,:,:,filter_index])

gradients返回的是张量列表，本表长度为1，因此保留第一个元素

In [12]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
# add this above
grads=tf.GradientTape.gradient(,loss,model.input)[0]
print(grads)
# grads=K.gradients(loss,model.input)[0]
# Use tf.GradientTape instead.

TypeError: gradient() missing 1 required positional argument: 'sources'

做除法之前加上1e-5，防止不小心除以0

利用keras后端函数实现（iterate）将numpy张量（长度为1的张量列表）
返回两个numpy的张量列表，两个张量分别是损失值和梯度值

In [ ]:
iterate = K.function([model.input],[loss,grads])

import numpy as np
loss_value, grads_value=iterate([np.zeros((1,150,150,3))])

从一张带有噪声的灰度图像开始
步长1，运行40次梯度上升，计算损失值和梯度，沿损失最大化方向调整

In [ ]:
input_img_data = np.random.random((1,150,150,3))*20+128.
step = 1.
for i in range(40):
    loss_value,grads_value = iterate([input_img_data])

    input_img_data += grads_value*step

要将张量浮点数转为[0，255]区间内的整数

In [ ]:
def deprocess_image(x):
    x-=x.mean()
    x/=(x.std()+1e-5)
    x*=0.1

    x+=0.5
    x=np.clip(x,0,1)

    x*=255
    x=np.clip(x,0,255).astype('uint8')
    return x

构建最大化损失函数

In [ ]:
def generate_pattern(layer_name,filter_index,size=150):
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:,:,:,filter_index])

    grads = K.gradients(loss,model.input)[0]

    grads/=(K.sqrt(K.mean(K.square(grads)))+1e-5)

    iterate =K.function([model.input],[loss,grads])

    input_img_data = np.random.random((1,size,size,3))*20+128.
    print(input_img_data)
    step = 1.
    for i in range(40):
        loss_value , grads_value = iterate([input_img_data])
        input_img_data +=grads_value*step

    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(generate_pattern('block3_conv1',0))